### Things to do

- ?
    - Official adaptation rate method
    - Hemisphere
    - Bin length is +.1 or *.1
- Analyze blocked and shuffled data in NCM and Field L
    - Check if there is a difference between NCM and Field L
    - Determine if there is a loss of decoding accuracy accross stimulus repetition
    - Hemispheric differences?
    - Analyze single unit data in a similar fashion
    - Figure out why part of shuffled data appears to be corrupted
   
- Analyze contrast data
    - Compare decoding accuracy between first and second half of presentations

In [ ]:
import mdlab as mdl
import dnlab as dnl
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import scipy.stats as sst
import pandas as pd
import pdb

In [ ]:
frcc=mdl.FRCC()
trailing_type = frcc.trailing_type
trailing_param = frcc.get_trailing_param(trailing_type)
stats_test = frcc.stats_test

In [ ]:
fn = mdl.get_pathname()

In [ ]:
fn2 = mdl.get_pathname()

In [ ]:
mdl.batch_mat2npz("hv_l_blocked.npz")

In [ ]:
header, spikes = mdl.load_npz_data(fn, False)
ncm_block = mdl.SpikeData(header, spikes)
ncm_mua_block=ncm_block.mua_or_sua("mua")
ncm_sep_mua_block = dnl.separate_npz_data(ncm_mua_block)
res_ncm_block = dnl.res_filter(ncm_sep_mua_block)

In [ ]:
header, spikes = mdl.load_npz_data(fn2, False)
l_block = mdl.SpikeData(header, spikes)
l_mua_block=l_block.mua_or_sua("mua")
l_sep_mua_block = dnl.separate_npz_data(l_mua_block)
res_l_block = dnl.res_filter(ncm_sep_mua_block)

In [ ]:
ncm_df = dnl.batch_decoding_accuracy(res_ncm_block)

In [ ]:
n =0
for ele in ncm_sep_mua_block[n].electrodes:
    ele_data = ncm_sep_mua_block[n].get_kw_SpikeData(electrode = ele)
    print mdl.birdid2str(ele_data.birdid[0])
    print ele
    if ele in res_ncm_block[n].electrodes:
        print "Responding"
    else:
        print "Not Responding"
    mdl.raster_plot(ele_data.spikes)
    plt.show()

In [17]:
z = res_ncm_block[0]

In [ ]:
header, spikes = mdl.load_npz_data('/Users/david/Documents/blocked.npz', False)
block = mdl.SpikeData(header, spikes)
#mua_block=block.mua_or_sua("mua")
sep_mua_block = dnl.separate_npz_data(block)

In [ ]:
res_block = dnl.res_filter(sep_mua_block)

In [ ]:
def site_check(site_data, frcc=frcc):
    trailing_type = frcc.trailing_type
    trailing_param = frcc.get_trailing_param(trailing_type)
    p_values = []
    for stim in site_data.stim:
        stim_data = site_data.get_kw_SpikeData(stim = stim)
        #baseline = stim_data.firing_rates(trailing_type, trailing_param, fr_type = 0)
        #stimulus = stim_data.firing_rates(trailing_type, trailing_param, fr_type = 1)
        #statistics, p_value = sst.wilcoxon(baseline, stimulus)
        fr = stim_data.firing_rates(trailing_type, trailing_param, fr_type = 2)
        base = np.zeros(len(fr))
        statistics, p_value = sst.ttest_rel(fr, base)
        #print baseline.shape
        p_values.append(p_value)
    return np.asarray(p_values)

In [ ]:
# loop through spike data
# loop through electrodes
# check if electrode is responding
# if responding, add header and spike
# combine at the end
def res_filter(split_data, p_value= 0.01):
    # to prevent enormous amount of warnings
    import warnings; warnings.simplefilter('ignore')
    res_split_data = []
    for spike_data in split_data:
        res_ele = []
        for ele in spike_data.electrodes:
            ele_spike_data = spike_data.get_kw_SpikeData(electrode=ele)
            respond = ele_spike_data.res_check()
            if respond["res"][0] < p_value:
                res_ele.append(ele)
        res_data = spike_data.get_kw_SpikeData(electrode = res_ele)
        res_split_data.append(res_data)
    return res_split_data

In [ ]:
# loop through spike data
# loop through electrodes
# check if electrode is responding
# if responding, add header and spike
# combine at the end
def res_filter2(split_data, p_value= 0.05):
    # to prevent enormous amount of warnings
    import warnings; warnings.simplefilter('ignore')
    res_split_data = []
    for spike_data in split_data:
        res_ele = []
        for ele in spike_data.electrodes:
            ele_spike_data = spike_data.get_kw_SpikeData(electrode=ele)
            respond = ele_spike_data.site_stats()
            if len(respond[respond["p"] < 0.05]) != 0:
                res_ele.append(ele)
        res_data = spike_data.get_kw_SpikeData(electrode = res_ele)
        res_split_data.append(res_data)
    return res_split_data

In [ ]:
# loop through spike data
# loop through electrodes
# check if electrode is responding
# if responding, add header and spike
# combine at the end
def res_filter(split_data, p_value= 0.05):
    # to prevent enormous amount of warnings
    import warnings; warnings.simplefilter('ignore')
    res_split_data = []
    for spike_data in split_data:
        res_ele = []
        for ele in spike_data.electrodes:
            ele_spike_data = spike_data.get_kw_SpikeData(electrode=ele)
            respond = site_check(ele_spike_data)
            r = respond < p_value
            if np.any(r) and not np.any(np.isnan((respond))):
                res_ele.append(ele)
        res_data = spike_data.get_kw_SpikeData(electrode = res_ele)
        res_split_data.append(res_data)
    return res_split_data

In [ ]:
n =0
for ele in sep_mua_block[n].electrodes:
    ele_data = sep_mua_block[n].get_kw_SpikeData(electrode = ele)
    print mdl.birdid2str(ele_data.birdid[0])
    print ele
    if ele in res_block[n].electrodes:
        print "Responding"
    else:
        print "Not Responding"
    mdl.raster_plot(ele_data.spikes)
    plt.show()

In [ ]:
# loop through spike data
# loop through electrodes
# check if electrode is responding
# if responding, add header and spike
# combine at the end
def res_filter(split_data, p_value= 0.05):
    #res_test = pd.DataFrame(columns=['birdid', 'electrode', 'Z'])
    res_header = pd.DataFrame()
    res_spikes = []
    res_split_data = []
    for spike_data in split_data:
        for ele in spike_data.electrodes:
            ele_spike_data = spike_data.get_kw_SpikeData(electrode=ele)
            respond = ele_spike_data.res_check()
            if respond["res"][0] < p_value:
                res_spikes.append(ele_spike_data.spikes)
                res_header = res_header.append(ele_spike_data.header)
        #return res_logidx
        res_spikes = np.vstack(res_spikes).tolist()
        #return res_header
        new_spike_data = mdl.SpikeData(res_header, np.asarray(res_spikes))
        res_split_data.append(new_spike_data)
    return res_split_data

In [ ]:
# calculates average adaptation rates for each site from trials 5-25 in a SpikeData (or BinnedData)
def get_adaptation_rate(SpikeData):
    adaptation_rates={}
    ele_arr=np.unique(SpikeData.header.electrode)
    stim_arr = np.unique(SpikeData.stim)
    for i in ele_arr:
        ele_data=SpikeData.get_kw_SpikeData(electrode=i)
        ele_rates=[]
        for j in stim_arr:
            stim_data=ele_data.get_kw_SpikeData(stim=j)
            fr=stim_data.firing_rates(trailing_type, trailing_param, fr_type = 1)
            ar=mdl.adaptation_rate(fr, robust=True)
            ele_rates.append(ar)
        adaptation_rates[i]=np.mean(ele_rates)
    return adaptation_rates

In [ ]:
# does the knn algorithm on an array of experiments
def batch_decoding_accuracy(split_data, n_neighbors=15, reps=250, training=.5):
    # Runs the decoding accuracy KNN algorithm on an array of spike data objects
    # split_data is the spike data array, bird_ele_class is a dataframe containing all responding electrodes and the 
    # region they are from.    
    # n neighbors is the amount of neighbors for the knn algorithm
    # reps is the amount of repitions of the algorithm to run
    # training is the training proportion
    
    c = ['birdid','electrode', 'hemisphere', 'average_adaptation_rate','average_decoding_accuracy','decoding_accuracy']
    final_df = pd.DataFrame(columns = c)
    #loops through split data
    n = 1
    for spike_data in split_data:
        ele_num_arr = spike_data.electrodes
        bird_tag = chr(int(str(spike_data.birdid[0])[0:2])) + chr(int(str(spike_data.birdid[0])[2:4]))
        bird_id = bird_tag + str(spike_data.birdid[0])[-5:-2]
        for x, ele_num in enumerate(ele_num_arr):
            ele_spike_data = spike_data.get_kw_SpikeData(electrode = ele_num)
            ele_binned_data = dnl.Spike_to_Binned(ele_spike_data)
            decoding_acc = dnl.decoding_accuracy(ele_binned_data, k=n_neighbors, rep_num=reps, training_prop=training)
            avg_decoding_acc = np.mean(decoding_acc)
            if ele_num < 1600:
                to_add = [bird_id, ele_num, "Left", dnl.get_adaptation_rate(ele_spike_data)[ele_num],avg_decoding_acc, decoding_acc]
                final_df = final_df.append(pd.DataFrame([to_add], columns=c)) 
            else:
                to_add = [bird_id, ele_num, "Right", dnl.get_adaptation_rate(ele_spike_data)[ele_num], avg_decoding_acc, decoding_acc]
                final_df = final_df.append(pd.DataFrame([to_add], columns=c))
        print str(n)+" Experiments Analyzed"
        n+=1
    return final_df.reset_index(drop=True)

In [22]:
y = z.get_kw_SpikeData(stim=1, electrode = 100)

In [30]:
z.header

,birdid,electrode,trial,stim,starting_time,ending_time,pre_stim,after_stim,resolution,filename
0,6676008.0,900,1,1,1.252152,1.701653,2.0,4.000060,0.0005,matrix_BL008__blocked_bird_.mat
1,6676008.0,900,2,1,6.252212,6.701713,2.0,4.000060,0.0005,matrix_BL008__blocked_bird_.mat
2,6676008.0,900,3,1,11.252272,11.701773,2.0,4.000060,0.0005,matrix_BL008__blocked_bird_.mat
3,6676008.0,900,4,1,16.252332,16.701833,2.0,4.000060,0.0005,matrix_BL008__blocked_bird_.mat
4,6676008.0,900,5,1,21.252392,21.701894,2.0,4.000060,0.0005,matrix_BL008__blocked_bird_.mat
5,6676008.0,900,6,1,26.252452,26.701953,2.0,4.000060,0.0005,matrix_BL008__blocked_bird_.mat
6,6676008.0,900,7,1,31.252512,31.702013,2.0,4.000060,0.0005,matrix_BL008__blocked_bird_.mat
7,6676008.0,900,8,1,36.252572,36.702073,2.0,4.000060,0.0005,matrix_BL008__blocked_bird_.mat
8,6676008.0,900,9,1,41.252632,41.702133,2.0,4.000060,0.0005,matrix_BL008__blocked_bird_.mat
9,6676008.0,900,10,1,46.252692,46.702193,2.0,4.000060,0.0005,matrix_BL008__blocked_bird_.mat


In [61]:
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split as split

In [110]:
test.bins.shape

(8320L, 64L)

In [108]:
z.spikes.shape

(8320L, 12000L)

In [112]:
def decoding_accuracy(electrode, k=15, rep_num=250, training_prop=.5):
    # Generates parameters for acc and trial_usage
    num_stim = len(electrode.stim)
    total_trials = electrode.header.shape[0]
    trials_per_stim = total_trials/num_stim
    acc = np.zeros((num_stim, trials_per_stim))
    
    #2D array #2D array, stim id x the trials per stimulus, increments up by one every time given trial used as target
    trial_usage = np.zeros((num_stim, trials_per_stim))
    
    indices = electrode.header.index.values
    stim_ids = np.array(electrode.header.stim)

    # knn algorithm
    for i in range(0,rep_num):
        try:
            #split data in half and predict values
            training_bins, target_bins, training_idx, target_idx, training_stim_ids, target_stim_ids = split(electrode.bins, indices, stim_ids, train_size=training_prop, test_size=1-training_prop)
            model = knn(n_neighbors=k, metric='euclidean')
            model.fit(training_bins, training_stim_ids)
            pred = model.predict(target_bins)
        except:
            pdb.set_trace()

        #Checks to see if predictions are correct
        for j in range(0, pred.shape[0]):
            stim_id= target_stim_ids[j]
            idx = target_idx[j]
            stim_data = electrode.get_kw_SpikeData(stim=stim_id)
            true_trial_num = stim_data.header.loc[idx:idx].trial.max()
            #pdb.set_trace()
            #increament trial usagebn
            trial_usage[stim_id-1][true_trial_num-1]+=1
            if pred[j] == stim_id:
                acc[stim_id-1][true_trial_num-1] += 1
            
    final_acc = np.divide(acc, trial_usage)
    return [final_acc, trial_usage]  

In [121]:
hund.get_kw_SpikeData(stim=4).header

,birdid,electrode,trial,stim,starting_time,ending_time,pre_stim,after_stim,resolution,filename
2480,6676008.0,100,241,4,1201.266552,1201.762947,2.0,4.00006,0.0005,matrix_BL008__blocked_bird_.mat
2481,6676008.0,100,242,4,1206.266612,1206.763007,2.0,4.00006,0.0005,matrix_BL008__blocked_bird_.mat
2482,6676008.0,100,243,4,1211.266675,1211.763070,2.0,4.00006,0.0005,matrix_BL008__blocked_bird_.mat
2483,6676008.0,100,244,4,1216.266732,1216.763127,2.0,4.00006,0.0005,matrix_BL008__blocked_bird_.mat
2484,6676008.0,100,245,4,1221.266795,1221.763190,2.0,4.00006,0.0005,matrix_BL008__blocked_bird_.mat
2485,6676008.0,100,246,4,1226.266852,1226.763247,2.0,4.00006,0.0005,matrix_BL008__blocked_bird_.mat
2486,6676008.0,100,247,4,1231.266912,1231.763307,2.0,4.00006,0.0005,matrix_BL008__blocked_bird_.mat
2487,6676008.0,100,248,4,1236.266972,1236.763367,2.0,4.00006,0.0005,matrix_BL008__blocked_bird_.mat
2488,6676008.0,100,249,4,1241.267032,1241.763427,2.0,4.00006,0.0005,matrix_BL008__blocked_bird_.mat
2489,6676008.0,100,250,4,1246.267092,1246.763487,2.0,4.00006,0.0005,matrix_BL008__blocked_bird_.mat


In [117]:
hundo = dnl.Spike_to_Binned(hund)

In [118]:
test_decoding = decoding_accuracy(hundo)

IndexError: index 242 is out of bounds for axis 0 with size 40

In [100]:
test_decoding

[array([[0.99230769, 0.99173554, 0.99206349, 0.99242424, 0.992     ,
         0.99230769, 0.99038462, 0.99315068, 0.99264706, 0.99264706,
         0.99193548, 0.992     , 0.99159664, 0.99212598, 0.99115044,
         0.99193548, 0.99285714, 0.99259259, 0.99242424, 0.99206349,
         0.99212598, 0.99180328, 0.99166667, 0.99107143, 0.99173554,
         0.99206349, 0.99193548, 0.99115044, 0.99280576, 0.99230769,
         0.99212598, 0.99224806, 0.99166667, 0.99122807, 0.99242424,
         0.99253731, 0.992     , 0.99173554, 0.99173554, 0.99242424]]),
 array([[130., 121., 126., 132., 125., 130., 104., 146., 136., 136., 124.,
         125., 119., 127., 113., 124., 140., 135., 132., 126., 127., 122.,
         120., 112., 121., 126., 124., 113., 139., 130., 127., 129., 120.,
         114., 132., 134., 125., 121., 121., 132.]])]

In [68]:
test.spikes.shape

(40L, 12000L)

In [ ]:
np.unique(np.array(res_block[0].header.stim))

In [ ]:
block_df = batch_decoding_accuracy(res_block)

In [18]:
z.stim

array([1, 2, 3, 4, 5, 6, 7, 8])